# Binary Classification - Bobcat

## Import Libraries for CNN

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os, shutil
import itertools
from keras import models
from keras.models import Model
from keras import layers
from sklearn.metrics import confusion_matrix, f1_score
np.random.seed(123)
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.applications import inception_v3
from keras.layers import Dense,GlobalAveragePooling2D

Using TensorFlow backend.


## Prepare Data

### Import, Resize, and Rescale Images

In [ ]:
data_te = ImageDataGenerator(rescale=1./255).flow_from_directory( 
        '/Users/j.markdaniels/Downloads/final_proj_data/bobcat_cougar_data/test/', 
        target_size=(224, 224), 
        batch_size = 340, 
        seed = 123)

In [ ]:
data_tr = ImageDataGenerator(rescale=1./255).flow_from_directory( 
        '/Users/j.markdaniels/Downloads/final_proj_data/bobcat_cougar_data/train/', 
        target_size=(224, 224), 
        batch_size = 340, 
        seed = 123) 

### Split Images and Labels into Arrays

In [ ]:
images_tr, labels_tr = next(data_tr)

In [ ]:
images_te, labels_te = next(data_te)

In [ ]:
images = np.concatenate((images_tr, images_te))

In [ ]:
labels = np.concatenate((labels_tr[:,0], labels_te[:,0]))

### Perform Train-Test Split

In [ ]:
from sklearn.model_selection import train_test_split
X_model, X_test, y_model, y_test = train_test_split(images, labels, test_size=0.20, random_state=123)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_model, y_model, test_size=0.20, random_state=123)

## Convolutional Neural Network (CNN)

### Create Model

In [ ]:
cnn = models.Sequential()
cnn.add(layers.Conv2D(64, (1, 1), activation='relu', input_shape=(224, 224,  3)))
cnn.add(layers.BatchNormalization())
cnn.add(layers.MaxPooling2D((2, 2)))
cnn.add(layers.Conv2D(64, (3, 3), activation='relu'))
cnn.add(layers.BatchNormalization())
# 64 bias parameters
# 64 * (3 * 3 * 3) weight parametrs
# Output is 64*224*224
cnn.add(layers.MaxPooling2D((2, 2)))
#Output is 64*112*112
cnn.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224,  3)))
cnn.add(layers.BatchNormalization())
#32 bias parameters
#32 * (3*3*64)
#Output is 32*112*112 
cnn.add(layers.MaxPooling2D((2, 2)))
cnn.add(layers.Flatten())
cnn.add(layers.Dense(32, activation='relu'))
cnn.add(layers.Dense(1, activation='sigmoid'))

cnn.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['acc'])

### Train Model

In [ ]:
cnn1 = cnn.fit(X_train,
                    y_train,
                    epochs=100,
                    batch_size=50,
                    validation_data=(X_val, y_val))

#### Summary

In [1]:
### Which should I keep? ###
print(cnn1.summary())
print(cnn.summary())

NameError: name 'cnn1' is not defined

### Evaluate CNN Model Performance

In [ ]:
hist_cnn = cnn1.history
loss_values = hist_cnn['loss']
val_loss_values = hist_cnn['val_loss']
acc_values = hist_cnn['acc'] 
val_acc_values = hist_cnn['val_acc']


epochs = range(1, len(loss_values) + 1)

plt.figure(figsize=(15,4))
plt.subplot(121)
plt.plot(epochs, loss_values, 'g.', label='Training loss')
plt.plot(epochs, val_loss_values, 'g', label='Validation loss')

plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(122)
plt.plot(epochs, acc_values, 'r.', label='Training acc')
plt.plot(epochs, val_acc_values, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

#### Accuracy

In [ ]:
results_train = cnn1.evaluate(X_train, y_train)
results_test = cnn1.evaluate(X_test, y_test)
print(results_train, results_test)

#### Confusion Matrix

In [ ]:
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [ ]:
plt.figure()
plot_confusion_matrix(confusion_matrix(y_test, predictions_transfer), classes=['bobcat', 'not bobcat'], normalize=False,
                      title='Confusion matrix - ImagenetV3')

#### F1 Score

In [ ]:
predictions_transfer = cnn1.predict(X_test)
predictions_transfer = np.around(predictions_transfer)

In [ ]:
f1_score(y_test, predictions_transfer)

### ROC Graph

In [ ]:
import numpy as np
import sklearn
from sklearn import metrics
sklearn.metrics.roc_curve(y_test, predictions_transfer, pos_label=None, sample_weight=None, drop_intermediate=True)

### Save Loaded CNN Model

In [ ]:
cnn.save('cnn1.h5')

### Load Saved CNN Model

In [ ]:
from numpy import loadtxt
from keras.models import load_model


cnn.load_weights('cnn1.h5')

model = load_model('cnn1.h5')

cnn1.summary()



## Convolutional Neural Network with Inception (CNN-i)

### Create Model

In [ ]:
imagenet=inception_v3.InceptionV3(weights='imagenet',include_top=False)
imagenet_new=imagenet.output
cnn_i = models.Sequential()
cnn_i.add(imagenet)
cnn_i.add(GlobalAveragePooling2D())
cnn_i.add(Dense(1024,activation='relu'))
cnn_i.add(Dense(1024,activation='relu')) #dense layer 2
cnn_i.add(Dense(512,activation='relu')) #dense layer 3
cnn_i.add(Dense(1,activation='sigmoid')) #final layer with sigmoid activation

In [ ]:
for i,layer in enumerate(imagenet.layers):
  print(i,layer.name, layer.trainable)

In [ ]:
for i,layer in enumerate(cnn_i.layers):
  print(i,layer.name, layer.trainable)

In [ ]:
for layer in cnn_i.layers[:1]:
    layer.trainable=False

In [ ]:
for i,layer in enumerate(cnn_i.layers):
  print(i,layer.name, layer.trainable)

### Train Model

In [ ]:
cnn_i.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

# step_size_train=train_generator.n//train_generator.batch_size
cnn_i.fit(X_train,
          y_train,
          epochs=100,
          batch_size=50,
          validation_data=(X_val, y_val))

### Evaluate Model

In [ ]:
hist_cnn = cnn_i.history
loss_values = hist_cnn['loss']
val_loss_values = hist_cnn['val_loss']
acc_values = hist_cnn['acc'] 
val_acc_values = hist_cnn['val_acc']


epochs = range(1, len(loss_values) + 1)

plt.figure(figsize=(15,4))
plt.subplot(121)
plt.plot(epochs, loss_values, 'g.', label='Training loss')
plt.plot(epochs, val_loss_values, 'g', label='Validation loss')

plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(122)
plt.plot(epochs, acc_values, 'r.', label='Training acc')
plt.plot(epochs, val_acc_values, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

#### Accuracy

In [ ]:
results_train = cnn_i.evaluate(X_train, y_train)
results_test = cnn_i.evaluate(X_test, y_test)
print(results_train, results_test)

#### Confusion Matrix

In [ ]:
predictions_transfer = cnn_i.predict(X_test)
predictions_transfer = np.around(predictions_transfer)

In [ ]:
plt.figure()
plot_confusion_matrix(confusion_matrix(y_test, predictions_transfer), classes=['bobcat', 'not bobcat'], normalize=False,
                      title='Confusion matrix - ImagenetV3')

#### F1 Score

In [ ]:
predictions_transfer = cnn_i.predict(X_test)
predictions_transfer = np.around(predictions_transfer)

In [ ]:
f1_score(y_test, predictions_transfer)

#### ROC Graph

In [ ]:
import numpy as np
import sklearn
from sklearn import metrics
sklearn.metrics.roc_curve(y_test, predictions_transfer, pos_label=None, sample_weight=None, drop_intermediate=True)

### Save Loaded CNN-i Model

In [ ]:
cnn_i.save('cnn_i.h5')

### Load Saved CNN-i Model

In [ ]:

from numpy import loadtxt
from keras.models import load_model


cnn_i.load_weights('cnn_i.h5')

model = load_model('cnn_i.h5')

cnn_i.summary()



# 20-class Wildlife Classification

## Import Libraries for Multiclass CNN with Inception

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os, shutil
import os
import numpy as np
import pandas as pd
import itertools
import seaborn as sns

from keras import models
from keras import layers
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.model_selection import train_test_split
from shutil import copy2

from keras.applications import inception_v3
from keras.layers import Dense,GlobalAveragePooling2D
from sklearn.metrics import confusion_matrix, f1_score
from keras.models import Model
from sklearn.model_selection import train_test_split


## Prepare Data

### Import, Resize, and Rescale Images

In [ ]:
data_te = ImageDataGenerator(rescale=1./255).flow_from_directory( 
        '/Users/j.markdaniels/Desktop/Data/multiclass/test/', 
        target_size=(224, 224), 
        batch_size = 2776, 
        class_mode='categorical',
        seed = 123)

data_tr = ImageDataGenerator(rescale=1./255).flow_from_directory( 
        '/Users/j.markdaniels/Desktop/Data/multiclass/train/', 
        target_size=(224, 224), 
        batch_size = 340, 
        class_mode='categorical',
        seed = 123) 

### Split Images and Labels into Arrays

In [ ]:
images_tr, labels_tr = next(data_tr)

In [ ]:
images_te, labels_te = next(data_te)

### Perform Train-Test Split

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(images_tr, labels_tr, test_size=0.20, random_state=123)

In [ ]:
y_test  = labels_te

In [ ]:
X_test = images_te

### Create Model

In [ ]:
imagenet=inception_v3.InceptionV3(weights='imagenet',include_top=False)
imagenet_new=imagenet.output
new_model = models.Sequential()
new_model.add(imagenet)
new_model.add(GlobalAveragePooling2D())
new_model.add(Dense(1024,activation='relu'))
new_model.add(Dense(1024,activation='relu')) #dense layer 2
new_model.add(Dense(512,activation='relu')) #dense layer 3
new_model.add(Dense(20,activation='softmax')) #final layer with softmax activation

In [ ]:
for i,layer in enumerate(imagenet.layers):
  print(i,layer.name, layer.trainable)

In [ ]:
for i,layer in enumerate(new_model.layers):
  print(i,layer.name, layer.trainable)

In [ ]:
for layer in new_model.layers[:1]:
    layer.trainable=False

In [ ]:
for i,layer in enumerate(new_model.layers):
  print(i,layer.name, layer.trainable)

### Train Model

In [ ]:
multi_class_model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

# step_size_train=train_generator.n//train_generator.batch_size
multi_class_model.fit(X_train,
          y_train,
          epochs=100,
          batch_size=60,
          validation_data=(X_val, y_val))

### Evaluate Model

#### Accuracy

In [ ]:
multi_class_model.evaluate(X_test, y_test)

#### Confusion Matrix

In [ ]:
predictions_transfer = multi_class_model.predict(X_test)

In [ ]:
y_pred = np.argmax(predictions_transfer, axis=1)

In [ ]:
y_true = np.where(y_test != 0)[1]

In [ ]:
labels = ['virginia_opossum','raccoon','coyote','red_fox','bald_eagle', 'seals', 'canada_lynx', 'elk', 'black_bear', 'raven', 'nutria', 'gray_fox', 'bobcat', 'ringtail', 'gray_wolf', 'mountain_beaver', 'sea_lions', 'deer', 'cougar', 'columbian_black-tailed_deer']
# Calculate Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
# classes = classes[unique_labels(y_true, y_pred)]
# Figure adjustment and heatmap plot
f = plt.figure(figsize=(20,30))
ax= plt.subplot()
sns.heatmap(cm, annot=True, ax = ax, vmax=100, cbar=False, cmap='Paired', mask=(cm==0), fmt=',.0f', linewidths=2, linecolor='grey', ); 

# labels
ax.set_xlabel('Predicted labels', fontsize=16);
ax.set_ylabel('True labels', labelpad=30, fontsize=16); 
ax.set_title('Confusion Matrix', fontsize=18); 
ax.xaxis.set_ticklabels(labels, rotation=90); 
ax.yaxis.set_ticklabels(labels, rotation=0);
ax.set_facecolor('white')

#### Top-k Categorical Accuracy

### Save Loaded Multi-class Model

In [ ]:
multi_class_model.save('inception_multiple_classes.h5')

### Load Saved Multi-class Model

In [ ]:
from numpy import loadtxt
from keras.models import load_model


cnn_i.load_weights('cnn_i.h5')

model = load_model('cnn_i.h5')

cnn_i.summary()
